# 1. 대화식 실행 환경의 준비
## A. (Version_1)
### a. mongodb 연결 및 데이터 확인

In [ ]:
from pymongo import MongoClient

mongodb_host = 'localhost'
mongodb_port = '27017'

mongo_client = MongoClient(mongodb_host, int(mongodb_port))
database = mongo_client.get_database('mydb')
collection = database['twitter']

result = collection.find()

# for i in range(100):
#     print(result[i])
    
# print(*collection.find({'lang':'ko'}), sep='\n')

for tweet in collection.find({'lang':'ko'}):
    if 'delete' in tweet:
        print(tweet)

### b. pandas로 보기 편하게 변환

In [ ]:
import pandas as pd
from pymongo import MongoClient

mongodb_host = 'localhost'
mongodb_port = '27017'

mongo_client = MongoClient(mongodb_host, int(mongodb_port))
database = mongo_client.get_database('mydb')
collection = database['twitter']

def tweets(*args, **kwargs):
    for tweet in collection.find(*args, **kwargs):
        if 'delete' not in tweet:
            # yield를 이용해 함수 외부로 값을 양보함
            yield{
                'created_at':tweet['created_at'], 'text':tweet['text']
            }
            
tweet = pd.DataFrame(tweets({'lang':'ko'}, limit=10))
tweet

## B. Version_2
### a. mongodb 연결 및 데이터 확인

In [5]:
import pandas as pd
import pymongo

# MongoDB에 접속
mongo = pymongo.MongoClient()

mongo.twitter.sample.find_one()

{'_id': ObjectId('61f9157ade1467ca4859aed6'),
 'created_at': 'Tue Feb 01 11:11:48 +0000 2022',
 'id': 1488470132406235137,
 'id_str': '1488470132406235137',
 'text': '"えちに自信ある人いる？★甘えん坊です',
 'source': '<a href="http://twittbot.net/" rel="nofollow">twittbot.net</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1327376936012640256,
  'id_str': '1327376936012640256',
  'name': '宮崎❦せふ募集沖縄',
  'screen_name': 'X06V9Kannroxiq8',
  'location': '沖縄',
  'url': None,
  'description': 'さみしがり屋の27歳 sex大好き セフレになってくれる人募集💓😘✨愛人募集犯されたい日もあるの。。こっちのことはなにも気にしないで性欲を吐き出してほしい気分😳',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 1,
  'friends_count': 68,
  'listed_count': 0,
  'favourites_count': 0,
  'statuses_count': 2092,
  'created_at': 'Fri Nov 13 22:25:05 +0000 2020',
  'utc_offset': None,
  'time_zone': No

### b. pandas로 보기 편하게 변환

In [1]:
import pandas as pd
import pymongo

mongo = pymongo.MongoClient()

# MongoDB로부터 레코드를 읽어들이는 제네레이터 함수
def tweets(*args, **kwargs):
    for tweet in mongo.twitter.sample.find(*args, **kwargs):
        if 'delete' not in tweet:
            # 흥미가 있는 필드만 yield한다.
            yield{
                'create_at': tweet['created_at'],
                'text': tweet['text'],
            }

# 제네레이터를 사용한 데이터 프레임 작성
pd.DataFrame(tweets({'lang': 'en'}, limit=2))

,create_at,text
0,Tue Feb 01 11:11:48 +0000 2022,RT @53Demon: It’s time for Morrison to man up ...
1,Tue Feb 01 11:11:48 +0000 2022,RT @TheRajivAdatia: Mere Bhai @kkundrra we nee...


# 2. etc..
## A. *args, **kwargs 실험

In [ ]:
def test(*args, **kwargs):
    # args에 tuple 형식으로 들어간다.
    print(args)
    # kwargs는 dict형식으로 들어간다.
    print(kwargs)
    
test({1:1}, s=10)